In [ ]:
import pandas as pd

def read_xyz_to_dataframe_no_frame_atom(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        
    traj_duration = 0
    i = 0
    all_atom_data = []
    while i < len(lines):
        # First line: number of atoms
        num_atoms = int(lines[i].strip())
        
        # Second line: general information (skip this line)
        general_info = lines[i + 1].strip()
        
        # Next num_atoms lines: atom data
        for j in range(num_atoms):
            atom_info = lines[i + 2 + j].strip().split()
            x, y, z = map(float, atom_info[1:])
            all_atom_data.append([x, y, z])
        
        # Move to the next block
        i += 2 + num_atoms
        traj_duration += 1
    
    df = pd.DataFrame(all_atom_data, columns=['X', 'Y', 'Z'])
    print(f"Trajectory duration: {traj_duration} frames")
    return df

# Test the function with the uploaded file
filename = './trajectory.xyz'
atom_data_df_no_frame_atom = read_xyz_to_dataframe_no_frame_atom(filename)
#print(atom_data_df_no_frame_atom)
df=atom_data_df_no_frame_atom

###################################################################################################################################################################

# Eliminazione della serie 'z' dal DataFrame
df.drop(columns=['Z'], inplace=True)
#df.drop(columns=['tSOAP'], inplace=True)

# Visualizzazione del DataFrame dopo l'eliminazione
#print("\nDataFrame dopo l'eliminazione della serie 'z':")
#df[['X', 'Y']] *= 1.4
#print(df)


###################################################################################################################################################################

import numpy as np

# Crea la serie 'particles'
num_particles = 6921
num_rows = len(df)
particle_series = np.arange(num_rows) % num_particles

# Aggiungi la serie 'particles' al DataFrame
df['particle'] = particle_series 

# Stampa il DataFrame risultante
#print(df)

###################################################################################################################################################################

# Aggiungi la colonna 'time'
df['time'] = (df['particle'] == 0).cumsum() - 1

# Visualizza il DataFrame risultante
#print(df)

###################################################################################################################################################################

# Calculate position differences for 'X' and 'Y'
df['dx'] = df.groupby('particle')['X'].diff()
df['dy'] = df.groupby('particle')['Y'].diff()

# Ensure 'time' differences are correct
df['dt'] = df.groupby('particle')['time'].diff()

# Compute velocities
df['vx'] = df['dx'] / df['dt']
df['vy'] = df['dy'] / df['dt']

# Handle NaN values resulting from the diff() operation
df['vx'].fillna(0, inplace=True)
df['vy'].fillna(0, inplace=True)

# Drop temporary columns used for calculations
df.drop(columns=['dx', 'dy', 'dt'], inplace=True)

# Print the final DataFrame
#print("\nFinal DataFrame with velocities:")
#print(df[['particle', 'time', 'X', 'Y', 'vx', 'vy']].head(50))

# Select a single particle to print its velocity pattern
particle_id = 0  # Change this to any particle ID you want to check
single_particle_df = df[df['particle'] == particle_id]

#print(f"\nVelocity pattern for particle {particle_id}:")
#print(single_particle_df[['time', 'X', 'Y', 'vx', 'vy']])

# Plotting velocity pattern for the selected particle
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Plot vx
plt.subplot(1, 2, 1)
plt.plot(single_particle_df['time'], single_particle_df['vx'], label='vx')
plt.xlabel('Time')
plt.ylabel('Velocity (vx)')
plt.title(f'Velocity (vx) Pattern for Particle {particle_id}')
plt.legend()

# Plot vy
plt.subplot(1, 2, 2)
plt.plot(single_particle_df['time'], single_particle_df['vy'], label='vy', color='orange')
plt.xlabel('Time')
plt.ylabel('Velocity (vy)')
plt.title(f'Velocity (vy) Pattern for Particle {particle_id}')
plt.legend()

plt.tight_layout()
plt.show()

###################################################################################################################################################################

# Reorder the columns
df = df[['X', 'Y', 'particle', 'time', 'vx', 'vy']]

M = df.to_numpy()

from scipy.spatial.distance import cdist

def calculate_fi(M, cutoff_radius=13.5):
    unique_times = np.unique(M[:, 3])
    fi = np.zeros(M.shape[0])
    count = 0
    for t in unique_times:
        mask = M[:, 3] == t
        M_same_time = M[mask]
        dist = cdist(M_same_time[:, :2],M_same_time[:, :2])
        
        
        for i in range(len(M_same_time)):
            v_i = M_same_time[i, 4:6]  # Velocità del punto i
            
            if np.linalg.norm(v_i) == 0.0:
                fi[count] = 0
                count +=1
                continue
            #dist_i = cdist(M_same_time[i, :2].reshape(1, -1), M_same_time[:, :2])[0]
            #dist_i = cdist(M_same_time[i, :2],M_same_time[:, :2]) #[0]
            
            neighbors = dist[i] <= cutoff_radius
            #print(np.sum(neighbors))
            if np.sum(neighbors) > 1:
                neighbor_vel = M_same_time[neighbors, 4:6]

                fi_i = 0

                for particle_j in M_same_time[neighbors]:
                
                    if np.all(particle_j == M_same_time[i]):
                        continue

                    v_j = particle_j[4:6]
                    if np.linalg.norm(v_j) == 0.0:
                        continue

                    fi_i += np.dot(v_j,v_i)/(np.linalg.norm(v_i)*np.linalg.norm(v_j))
                
                    #fi[mask][i] = fi_i
                fi_i /= (np.sum(neighbors) -1)
                fi[count] = fi_i
                count +=1
            else : 
                fi[count] = 0
                count +=1  
    return fi

M = M  

fi = calculate_fi(M,cutoff_radius=13.5)

df['fi'] = fi

M_with_fi = np.column_stack((M, fi))

df_sorted = df.sort_values(by=['particle', 'time'])

#Reshape fi into a matrix
num_particles = 6921
traj_duration = df_sorted['time'].max() + 1  # Since time starts from 0

fi_matrix = df_sorted['fi'].values.reshape((num_particles, traj_duration))

print(fi_matrix.shape)


#print("Matrice M con la nuova colonna fi:")
#print(M_with_fi)

###################################################################################################################################################################

import matplotlib.pyplot as plt

# Velocity Distribution
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(df['vx'], bins=50, alpha=0.7, label='vx')
plt.xlabel('Velocity (vx)')
plt.ylabel('Frequency')
plt.title('Velocity Distribution (vx)')
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(df['vy'], bins=50, alpha=0.7, label='vy', color='orange')
plt.xlabel('Velocity (vy)')
plt.ylabel('Frequency')
plt.title('Velocity Distribution (vy)')
plt.legend()

plt.tight_layout()
plt.show()

# Fi Distribution
plt.figure(figsize=(6, 6))
plt.hist(df['fi'], bins=50, alpha=0.7, color='green')
plt.xlabel('Fi')
plt.ylabel('Frequency')
plt.title('Distribution of Fi')
plt.show()

# Calculate the magnitude of the velocities
velocity_magnitude = np.sqrt(df['vx']**2 + df['vy']**2)

# Filter out low magnitude velocities
mask = velocity_magnitude >= 0.01
filtered_df = df[mask]
filtered_velocity_magnitude = velocity_magnitude[mask]

# Sort the filtered DataFrame by velocity magnitude to plot higher velocities on top
sorted_indices = np.argsort(filtered_velocity_magnitude)
sorted_df = filtered_df.iloc[sorted_indices]
sorted_velocity_magnitude = filtered_velocity_magnitude.iloc[sorted_indices]

plt.figure(figsize=(12, 8))

# Create the quiver plot with color coding
quiver = plt.quiver(
    sorted_df['X'], sorted_df['Y'],
    sorted_df['vx'], sorted_df['vy'],
    sorted_velocity_magnitude, angles='xy', scale_units='xy', scale=0.1, cmap='viridis', linewidth=0.5
)

# Add a color bar
cbar = plt.colorbar(quiver)
cbar.set_label('Velocity Magnitude')

plt.xlabel('X')
plt.ylabel('Y')
plt.title('Enhanced Velocity Field')
plt.show()

# Fi Over Time
plt.figure(figsize=(12, 6))
plt.plot(df['time'], df['fi'], 'bo', markersize=2)
plt.xlabel('Time')
plt.ylabel('Fi')
plt.title('Fi Over Time')
plt.show()

# Average Fi Per Time Step
average_fi_per_time = df.groupby('time')['fi'].mean()

plt.figure(figsize=(12, 6))
plt.plot(average_fi_per_time.index, average_fi_per_time.values, 'r-')
plt.xlabel('Time')
plt.ylabel('Average Fi')
plt.title('Average Fi Per Time Step')
plt.show()

particle_id = 0
fi_trend_id_0 = df[df['particle'] == particle_id]['fi']

# Plot the fi trend for particle ID 0
plt.figure(figsize=(10, 6))
plt.plot(fi_trend_id_0.values, label=f'Particle ID {particle_id}', color='red', linewidth=2)
plt.xlabel('Time')
plt.ylabel('Fi')
plt.title(f'Fi Trend for Particle ID {particle_id}')
plt.legend()

plt.show()

# Plot the fi trend for particle ID 0
plt.figure(figsize=(10, 6))
plt.plot(fi_matrix[particle_id,:], label=f'Particle ID {particle_id}', color='red', linewidth=2)
plt.xlabel('Time')
plt.ylabel('Fi')
plt.title(f'Fi Trend for Particle ID {particle_id}')
plt.legend()

plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# Function to read XYZ data
def read_xyz_to_dataframe_no_frame_atom(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        
    traj_duration = 0
    i = 0
    all_atom_data = []
    while i < len(lines):
        # First line: number of atoms
        num_atoms = int(lines[i].strip())
        
        # Second line: general information (skip this line)
        general_info = lines[i + 1].strip()
        
        # Next num_atoms lines: atom data
        for j in range(num_atoms):
            atom_info = lines[i + 2 + j].strip().split()
            x, y, z = map(float, atom_info[1:])
            all_atom_data.append([x, y, z])
        
        # Move to the next block
        i += 2 + num_atoms
        traj_duration += 1
    
    df = pd.DataFrame(all_atom_data, columns=['X', 'Y', 'Z'])
    return df, traj_duration

# Function to calculate fi
def calculate_fi(M, cutoff_radius):
    unique_times = np.unique(M[:, 3])
    fi = np.zeros(M.shape[0])
    count = 0
    for t in unique_times:
        mask = M[:, 3] == t
        M_same_time = M[mask]
        dist = cdist(M_same_time[:, :2], M_same_time[:, :2])
        
        for i in range(len(M_same_time)):
            v_i = M_same_time[i, 4:6]  # Velocità del punto i
            
            if np.linalg.norm(v_i) == 0.0:
                fi[count] = 0
                count += 1
                continue
            
            neighbors = dist[i] <= cutoff_radius
            if np.sum(neighbors) > 1:
                neighbor_vel = M_same_time[neighbors, 4:6]
                fi_i = 0
                for particle_j in M_same_time[neighbors]:
                    if np.all(particle_j == M_same_time[i]):
                        continue
                    v_j = particle_j[4:6]
                    if np.linalg.norm(v_j) == 0.0:
                        continue
                    fi_i += np.dot(v_j, v_i) / (np.linalg.norm(v_i) * np.linalg.norm(v_j))
                fi_i /= (np.sum(neighbors) - 1)
                fi[count] = fi_i
                count += 1
            else:
                fi[count] = 0
                count += 1  
    return fi

# Read the file and prepare initial DataFrame
filename = './trajectory.xyz'
df, traj_duration = read_xyz_to_dataframe_no_frame_atom(filename)

# Drop Z column
df.drop(columns=['Z'], inplace=True)

# Create particle series and add to DataFrame
num_particles = 6921
num_rows = len(df)
particle_series = np.arange(num_rows) % num_particles
df['particle'] = particle_series

# Add time column
df['time'] = (df['particle'] == 0).cumsum() - 1

# Calculate position differences and velocities
df['dx'] = df.groupby('particle')['X'].diff()
df['dy'] = df.groupby('particle')['Y'].diff()
df['dt'] = df.groupby('particle')['time'].diff()
df['vx'] = df['dx'] / df['dt']
df['vy'] = df['dy'] / df['dt']
df['vx'].fillna(0, inplace=True)
df['vy'].fillna(0, inplace=True)
df.drop(columns=['dx', 'dy', 'dt'], inplace=True)

# Reorder the columns
df = df[['X', 'Y', 'particle', 'time', 'vx', 'vy']]
M = df.to_numpy()

# List of cutoff values
cutoff_list = [80,100,120,140,180,200]
colors = ['red', 'blue', 'green', 'orange', 'purple','yellow']

plt.figure(figsize=(12, 6))

# Iterate over cutoff values
for cutoff, color in zip(cutoff_list, colors):
    fi = calculate_fi(M, cutoff_radius=cutoff)
    df['fi'] = fi
    df_sorted = df.sort_values(by=['particle', 'time'])
    fi_matrix = df_sorted['fi'].values.reshape((num_particles, traj_duration))
    fi_trend_id_0 = df[df['particle'] == 0]['fi']
    np.savez(f'wave_PHI_cutoff_{cutoff}A.npz', fi_matrix=fi_matrix)
    # Plot the fi trend for particle ID 0
    print(cutoff)
    plt.plot(fi_trend_id_0.values, label=f'Cutoff {cutoff}', color=color, linewidth=2)

plt.xlabel('Time')
plt.ylabel('Fi')
plt.title('Fi Trend for Particle ID 0 for Different Cutoff Radii')
plt.legend()
plt.show()


In [ ]:
import pandas as pd

def read_xyz_to_dataframe_no_frame_atom(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        
    traj_duration = 0
    i = 0
    all_atom_data = []
    while i < len(lines):
        # First line: number of atoms
        num_atoms = int(lines[i].strip())
        
        # Second line: general information (skip this line)
        general_info = lines[i + 1].strip()
        
        # Next num_atoms lines: atom data
        for j in range(num_atoms):
            atom_info = lines[i + 2 + j].strip().split()
            x, y, z = map(float, atom_info[1:])
            all_atom_data.append([x, y, z])
        
        # Move to the next block
        i += 2 + num_atoms
        traj_duration += 1
    
    df = pd.DataFrame(all_atom_data, columns=['X', 'Y', 'Z'])
    print(f"Trajectory duration: {traj_duration} frames")
    return df

# Test the function with the uploaded file
filename = './trajectory.xyz'
atom_data_df_no_frame_atom = read_xyz_to_dataframe_no_frame_atom(filename)
#print(atom_data_df_no_frame_atom)
df=atom_data_df_no_frame_atom

###################################################################################################################################################################

# Eliminazione della serie 'z' dal DataFrame
df.drop(columns=['Z'], inplace=True)

###################################################################################################################################################################
import numpy as np

# Crea la serie 'particles'
num_particles = 6921
num_rows = len(df)
particle_series = np.arange(num_rows) % num_particles

# Aggiungi la serie 'particles' al DataFrame
df['particle'] = particle_series 

###################################################################################################################################################################

# Aggiungi la colonna 'time'
df['time'] = (df['particle'] == 0).cumsum() - 1

###################################################################################################################################################################

# Calculate position differences for 'X' and 'Y'
df['dx'] = df.groupby('particle')['X'].diff()
df['dy'] = df.groupby('particle')['Y'].diff()

# Ensure 'time' differences are correct
df['dt'] = df.groupby('particle')['time'].diff()

# Compute velocities
df['vx'] = df['dx'] / df['dt']
df['vy'] = df['dy'] / df['dt']

# Handle NaN values resulting from the diff() operation
df['vx'].fillna(0, inplace=True)
df['vy'].fillna(0, inplace=True)

# Drop temporary columns used for calculations
df.drop(columns=['dx', 'dy', 'dt'], inplace=True)

# Select a single particle to print its velocity pattern
particle_id = 0  # Change this to any particle ID you want to check
single_particle_df = df[df['particle'] == particle_id]

#print(f"\nVelocity pattern for particle {particle_id}:")
#print(single_particle_df[['time', 'X', 'Y', 'vx', 'vy']])

# Plotting velocity pattern for the selected particle
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Plot vx
plt.subplot(1, 2, 1)
plt.plot(single_particle_df['time'], single_particle_df['vx'], label='vx')
plt.xlabel('Time')
plt.ylabel('Velocity (vx)')
plt.title(f'Velocity (vx) Pattern for Particle {particle_id}')
plt.legend()

# Plot vy
plt.subplot(1, 2, 2)
plt.plot(single_particle_df['time'], single_particle_df['vy'], label='vy', color='orange')
plt.xlabel('Time')
plt.ylabel('Velocity (vy)')
plt.title(f'Velocity (vy) Pattern for Particle {particle_id}')
plt.legend()

plt.tight_layout()
plt.show()

###################################################################################################################################################################

# Reorder the columns
df = df[['X', 'Y', 'particle', 'time', 'vx', 'vy']]

M = df.to_numpy()

from scipy.spatial.distance import cdist

df_sorted = df.sort_values(by=['particle', 'time'])

###################################################################################################################################################################

import matplotlib.pyplot as plt

# Velocity Distribution
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(df['vx'], bins=50, alpha=0.7, label='vx')
plt.xlabel('Velocity (vx)')
plt.ylabel('Frequency')
plt.title('Velocity Distribution (vx)')
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(df['vy'], bins=50, alpha=0.7, label='vy', color='orange')
plt.xlabel('Velocity (vy)')
plt.ylabel('Frequency')
plt.title('Velocity Distribution (vy)')
plt.legend()

plt.tight_layout()
plt.show()


# Calculate the magnitude of the velocities
velocity_magnitude = np.sqrt(df['vx']**2 + df['vy']**2)

# Filter out low magnitude velocities
mask = velocity_magnitude >= 0.01
filtered_df = df[mask]
filtered_velocity_magnitude = velocity_magnitude[mask]

# Sort the filtered DataFrame by velocity magnitude to plot higher velocities on top
sorted_indices = np.argsort(filtered_velocity_magnitude)
sorted_df = filtered_df.iloc[sorted_indices]
sorted_velocity_magnitude = filtered_velocity_magnitude.iloc[sorted_indices]

plt.figure(figsize=(8, 5),dpi=300)

# Create the quiver plot with color coding
quiver = plt.quiver(
    sorted_df['X'], sorted_df['Y'],
    sorted_df['vx'], sorted_df['vy'],
    sorted_velocity_magnitude, angles='xy', scale_units='xy', scale=0.1, cmap='viridis', linewidth=0.5
)

# Add a color bar
cbar = plt.colorbar(quiver)
cbar.set_label('Velocity Magnitude')

plt.xlabel('X')
plt.ylabel('Y')
plt.title('Enhanced Velocity Field')
plt.subplots_adjust(left=0.2, right=0.8, top=0.8, bottom=0.2)
plt.show()
